(In order to load the stylesheet of this notebook, execute the last code cell in this notebook)

# Stack Overflow

## Introduction 

In this assignment, we will look at some posts on Stack Overflow during the year of 2015 and measure the similarity of users by looking at the types of questions they answer. We will also analyze the creation dates of questions.

## Step 0. Preparation

Before we start working on the notebook, let's make sure that everything is setup properly. You should have downloaded and installed
* [Anaconda](https://store.continuum.io/cshop/anaconda/)
* [Git](http://git-scm.com/downloads)

If you are working from the undergraduate lab (on a linux machine) these are both installed, but you need to follow the instructions [from here](https://github.com/datascience16/lectures/blob/master/Lecture2/Getting-Started.ipynb).



## Step 1. Getting the data

Let's make a sample request to retrieve the questions posted on Stack Exchange on the first day of 2015. Documentation of the Stack Exchange API can be found [here](https://api.stackexchange.com/docs).

In [1]:
import requests

start_time = 1420070400 # 01-01-2015 at 00:00:00
end_time   = 1420156800 # 01-02-2015 at 00:00:00

response = requests.get("https://api.stackexchange.com/2.2/questions?pagesize=100" +
                        "&fromdate=" + str(start_time) + "&todate=" + str(end_time) +
                        "&order=asc&sort=creation&site=stackoverflow")
print response

<Response [200]>


All dates in the Stack Exchange API are in [unix epoch time](https://en.wikipedia.org/wiki/Unix_time). The format for the request string is specified [here](https://api.stackexchange.com/docs/questions).

We can try to print the response that Stack Exchange returns.

In [2]:
print response.text

{"items":[{"tags":["php","mysql","arrays","mysqli"],"owner":{"reputation":157,"user_id":3210431,"user_type":"registered","accept_rate":78,"profile_image":"https://i.stack.imgur.com/QTjBV.jpg?s=128&g=1","display_name":"salep","link":"http://stackoverflow.com/users/3210431/salep"},"is_answered":true,"view_count":198,"accepted_answer_id":27727478,"answer_count":2,"score":0,"last_activity_date":1420071815,"creation_date":1420070458,"question_id":27727385,"link":"http://stackoverflow.com/questions/27727385/inserting-multiple-records-into-mysql-from-an-array","title":"Inserting multiple records into MySQL from an array"},{"tags":["apache-pig"],"owner":{"reputation":6,"user_id":868779,"user_type":"registered","profile_image":"https://www.gravatar.com/avatar/19f7315b44e97ca1bc069694fdba7428?s=128&d=identicon&r=PG","display_name":"Balpreet Pankaj","link":"http://stackoverflow.com/users/868779/balpreet-pankaj"},"is_answered":false,"view_count":222,"answer_count":1,"score":1,"last_activity_date":

It is not possible to read the raw response. Instead, we need to decode the raw response as JSON and use the `json` library to print it.

In [3]:
import json
#print json.loads(json.dumps(response.json(), indent=2))
print json.dumps(response.json(), indent=2)

{
  "has_more": true, 
  "items": [
    {
      "is_answered": true, 
      "view_count": 198, 
      "tags": [
        "php", 
        "mysql", 
        "arrays", 
        "mysqli"
      ], 
      "last_activity_date": 1420071815, 
      "answer_count": 2, 
      "creation_date": 1420070458, 
      "score": 0, 
      "link": "http://stackoverflow.com/questions/27727385/inserting-multiple-records-into-mysql-from-an-array", 
      "accepted_answer_id": 27727478, 
      "owner": {
        "user_id": 3210431, 
        "profile_image": "https://i.stack.imgur.com/QTjBV.jpg?s=128&g=1", 
        "user_type": "registered", 
        "reputation": 157, 
        "link": "http://stackoverflow.com/users/3210431/salep", 
        "accept_rate": 78, 
        "display_name": "salep"
      }, 
      "title": "Inserting multiple records into MySQL from an array", 
      "question_id": 27727385
    }, 
    {
      "is_answered": false, 
      "view_count": 222, 
      "tags": [
        "apache-pig"
      

Now we can easily see that the response consists of a list of question items. For each of these items, we get information about its attributes such as its `creation_date`, `answer_count`, `owner`, `title`, etc.

Notice that has_more is true. To get more items, we can [request the next page](https://api.stackexchange.com/docs/paging).

-----------------

## Step 2. Parsing the responses

In this section, we practice some of the basic Python tools that we learned in class and the powerful string handling methods that Python offers. Our goal is to be able to pick the interesting parts of the response and transform them in a format that will be useful to us.

First let's isolate the creation_date in the response. Fill in the rest of the ```print_creation_dates_json()``` function that reads the response and prints the creation dates. Notice that a JSON object is basically a dictionary. **(5 pts)**

In [4]:
def print_creation_dates_json(response):
    response = json.dumps(response.json(), indent=2)
    jsonRes = json.loads(response)
    data = jsonRes["items"]
    hasmore = jsonRes["has_more"]
    
    while hasmore:
        for d in data:
            creation_date = d.get("creation_date")
            print creation_date
    

Write the code that calls the ```print_creation_dates_json()``` function to print out all the creation dates of questions posted on the first day in 2015. Please be aware of Stack Exchange's [rate limit](https://api.stackexchange.com/docs/throttle). **(5 pts)**

In [ ]:
print_creation_dates_json(response)

Due to time constraints, we have downloaded the [data dump](http://cs-people.bu.edu/kzhao/teaching/stackoverflow-posts-2015.tar.gz) for Stack Overflow's posts in 2015. Note that this file is 10GB. If you don't have space on your computer, you can download it into `/scratch` on one of the machines in the undergrad lab or you can download it onto a USB. You may also want to work with a subset of this data at first, but your solution should be efficient enough to work with the whole dataset. For example, if you call `read()` on this file, you will get a `MemoryError`.

Write a function to parse out the questions posted in 2015. These are posts with `PostTypeId=1`. Make a `pandas DataFrame` with 4 columns: `Id`, `CreationDate`, `OwnerUserId`, and the first tag in `Tags`. Save the `DataFrame` to a file named `question_dataframe.csv`. **(10 pts)**

In [7]:
import xml.etree.ElementTree as etree
import pandas as pd
def parseout(xmlfile):
    Id = []
    CreationDate = []
    OwnerUserId = []
    FirstTag = []
    for event, elem in etree.iterparse(xmlfile,events = ('start', 'end', 'start-ns', 'end-ns')):
        if event == "start":
            if elem.tag == "row":
                postTypeId = elem.get("PostTypeId")
                if postTypeId == "1":
                    Id.append(elem.get("Id"))
                    CreationDate.append(elem.get("CreationDate"))
                    OwnerUserId.append(elem.get("OwnerUserId"))
                    tags = elem.get("Tags")
                    if tags[0] =="<":
                        l = tags.index(">")
                        tags = tags[1:l]
                    FirstTag.append(tags)
                        
    data = {"Id": Id,
           "CreationDate" : CreationDate,
           "OwnerUserId" : OwnerUserId,
           "FirstTag" : FirstTag}
    
    frame = pd.DataFrame(data)
    frame.to_csv("question_dataframe.csv")
                
            
            
            

In [8]:
import pandas as pd
k = open("/Users/johnsakk/Desktop/stackoverflow-posts-2015-subset.xml", "r")
parseout(k)

question_df = pd.read_csv("question_dataframe.csv")
print question_df

     Unnamed: 0             CreationDate               FirstTag        Id  \
0             0  2015-01-01T00:00:58.253                    php  27727385   
1             1  2015-01-01T00:01:43.673             apache-pig  27727388   
2             2  2015-01-01T00:02:32.123                    ios  27727391   
3             3  2015-01-01T00:02:57.983             sql-server  27727393   
4             4  2015-01-01T00:03:31.337                    php  27727394   
5             5  2015-01-01T00:04:01.407                android  27727396   
6             6  2015-01-01T00:05:03.773                     c#  27727406   
7             7  2015-01-01T00:05:27.167                   java  27727407   
8             8  2015-01-01T00:05:34.733                     c#  27727408   
9             9  2015-01-01T00:06:17.720                 apache  27727409   
10           10  2015-01-01T00:06:41.067                  mysql  27727410   
11           11  2015-01-01T00:07:28.747             javascript  27727414   

-----------------

## Step 3. Putting it all together

We are now ready to tackle our original problem. Write a function to measure the similarity of the top 1000 users with the most answer posts. Compare the users based on the types of questions they answer. We will categorize the questions by looking at the first tag in each question. You may choose to implement any one of the similarity/distance measures we discussed in class. Document your findings. **(30pts)**

Note that answers are posts with `PostTypeId=2`. The ID of the question in answer posts is the `ParentId`.

You may find the [sklearn.feature_extraction module](http://scikit-learn.org/stable/modules/feature_extraction.html) helpful.

In [148]:
import pandas
import xml.etree.ElementTree as etree
from operator import itemgetter
print "hi"
def topthou(xmlfile):
    users= []
    CurrUserScore=0
    CurrUser = []
    UserRank= []
    topusers = []
    triaAtt =[]
    tagit=[]
    tpl = []
    finl=[]
    UserProfile=[]
    for event, elem in etree.iterparse(xmlfile,events = ('start', 'end', 'start-ns', 'end-ns')):
        if event == "start":
            if elem.tag == "row":
                postTypeId = elem.get("PostTypeId")
                if postTypeId == "2":
                    parentId = elem.get("ParentId")
                    userId = str(elem.get("OwnerUserId"))
                    if userId != "None":
                      #  userId = str(elem.get("OwnerUserId"))
                        users.append(userId)
                        triaAtt.append([userId, str(parentId)])
                elif postTypeId == "1":
                    tags = elem.get("Tags")
                    iD = elem.get("Id")
                    if tags[0] =="<":
                        l = tags.index(">")
                        tags = tags[1:l]
                        tagit.append([iD,tags])
    

    f = 0                    
    users = sorted(users, key=int)
    for i in users:
        if( i != None):
                CurrUser = i
                for j in users:
                    f=f+1
                    if  f == 1:
                        UserProfile = [CurrUser, 0]
                        UserRank.append(UserProfile)
                    elif (CurrUser == (UserRank[-1])[0]):
                        (UserRank[-1])[1] = (UserRank[-1])[1]+1
                    else:
                        UserRank.append([CurrUser, 0])
                
    UserRank = sorted(UserRank, key = itemgetter(1))
    UserRank = UserRank[::-1]
    UserRank = UserRank[:999]
    print UserRank
    g=0
    if [item for item in UserRank if item[0]][g] == [item for item in triaAtt if item[0]][g]:
        g=g+1
        tpl.append([[item for item in k if item[0]][g], [item for item in l if item[1]][g]])
            
    
    for k in tpl:
        for l in tagit:
            if [item for item in k if item[1]] == [item for item in l if item[0]]:
                finl.append([[item for item in k if item[0]], [item for item in l if item[1]]])
            
    list         
    finl = sorted(finl, key = itemgetter(0))
    
    
    
                        
    
    
        
                        



hi


In [149]:
import pandas
import xml.etree.ElementTree as etree
k = open("/Users/johnsakk/Desktop/stackoverflow-posts-2015-subset.xml", "r")

topthou(k)

[['963195', 3467], ['4409544', 2889], ['3808877', 2889], ['2494262', 2889], ['1491895', 2889], ['4409568', 2311], ['3849456', 2311], ['3349551', 2311], ['3113558', 2311], ['2224577', 2311], ['794088', 2311], ['4171355', 1733], ['4101237', 1733], ['2946873', 1733], ['2840591', 1733], ['2684539', 1733], ['2225682', 1733], ['2142582', 1733], ['2058037', 1733], ['1935918', 1733], ['1858561', 1733], ['1855677', 1733], ['1455003', 1733], ['1285133', 1733], ['1143874', 1733], ['992484', 1733], ['541091', 1733], ['511529', 1733], ['207421', 1733], ['20862', 1733], ['4354956', 1155], ['4326278', 1155], ['4254562', 1155], ['4224089', 1155], ['4059625', 1155], ['3973077', 1155], ['3925941', 1155], ['3829407', 1155], ['3724237', 1155], ['3575886', 1155], ['3422102', 1155], ['3385827', 1155], ['3379653', 1155], ['3225108', 1155], ['3135317', 1155], ['3016063', 1155], ['2959554', 1155], ['2951037', 1155], ['2745485', 1155], ['2541095', 1155], ['2110412', 1155], ['1940592', 1155], ['1762224', 1155], 

Let's plot a subset of the distance matrix. Order the pairwise distance in your distance matrix (excluding the entries along the diagonal) in increasing order and pick user pairs until you have 100 unique users. See [Lecture 3](https://github.com/datascience16/lectures/blob/master/Lecture3/Distance-Functions.ipynb) for examples. **(10 pts)**

Next, let's create some time series from the data. Look at the top 100 users with the most question posts. For each user, your time series will be the `CreationDate` of the questions posted by that user. You may want to make multiple time series for each user based on the first tag of the questions. Compare the time series using one of the methods discussed in class. Document your findings. **(30 pts)**

You may find the [pandas.DataFrame.resample module](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html) helpful.

Plot the 2 most similar and the 2 most different time series. **(10 pts)**

In [ ]:
# Code for setting the style of the notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("../theme/custom.css", "r").read()
    return HTML(styles)
css_styling()